## **STAGE-1: BINARY CLASSIFIER** 
**Recyclable vs Non-Recyclable**

using TensorFlow + Transfer Learning (MobileNetV2).



In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "binary_dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_data = val_test_gen.flow_from_directory(
    "binary_dataset/val",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_data = val_test_gen.flow_from_directory(
    "binary_dataset/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)


Found 5198 images belonging to 2 classes.
Found 1110 images belonging to 2 classes.
Found 1123 images belonging to 2 classes.


### **MODEL ARCHITECTURE**

#### Input Image (224×224×3)
↓
#### MobileNetV2 (Frozen)
↓
#### GlobalAveragePooling
↓
#### Dense (128, ReLU)
↓
#### Dropout (0.5)
↓
#### Dense (1, Sigmoid)


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

IMG_SIZE = (224, 224)

base_model = MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)

# Freeze base model
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [3]:
EPOCHS = 10

history = model.fit(
    train_data,
    validation_data=val_data, # we used val data to monitor the performace of training.
    epochs=EPOCHS
)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 43s 253ms/step - accuracy: 0.9167 - loss: 0.2082 - val_accuracy: 0.9631 - val_loss: 0.1051
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 41s 253ms/step - accuracy: 0.9533 - loss: 0.1265 - val_accuracy: 0.9676 - val_loss: 0.0910
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 43s 261ms/step - accuracy: 0.9608 - loss: 0.1076 - val_accuracy: 0.9631 - val_loss: 0.0893
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 44s 268ms/step - accuracy: 0.9677 - loss: 0.0906 - val_accuracy: 0.9685 - val_loss: 0.0835
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 44s 272ms/step - accuracy: 0.9681 - loss: 0.0813 - val_accuracy: 0.9649 - val_loss: 0.0784
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 274ms/step - accuracy: 0.9736 - loss: 0.0755 - val_accuracy: 0.9667 - val_loss: 0.0893
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 45s 276ms/step - accuracy: 0.9721 - loss: 0.0705 - val_accuracy: 0.9640 - val_loss: 0.1154
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 44s 271ms/step - accuracy: 0.9773 - loss: 0

## **EVALUATE** ON TEST SET

In [4]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")


36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 192ms/step - accuracy: 0.9724 - loss: 0.0894
Test Accuracy: 0.9724


In [5]:
model.save("binary_waste_classifier.h5")
print("✅ Binary model saved successfully!")


✅ Binary model saved successfully!


### **model daigniosis** 
- Train accuracy ≈ 97–98%
- Validation accuracy ≈ 96–97%
- Validation loss stable (≈ 0.08–0.11)
- test accuracy ≈ train accuracy (model generalizes well) 

Model is learning well, No severe overfitting
⚠️ Small train–val gap → fine-tuning may give 1–2% gain
## Fine tuning 
unfreeze last 20 layers.

### **ways can lead to catastrophic forgetting**
When you unfreeze too many layers.
Gradients now flow into pretrained layers,
Even small updates can change learned features

also :
- Large LR → big weight changes → destroy pretrained features
- Small LR → gentle adaptation to our dataset images.


also if we train too long(more epochs):
-  The model forgets pre-trained general features
-  It overfits to our small dataset 
- Validation loss increases

This are the ways leads to **catastrophic forgetting.**

snice our dataset is small compared to ImageNet, so:
The model adapts quickly so therby i used 
- Few epochs + low LR = safe fine-tuning 🏁

In [6]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

# re-compile with low learning rate to avoid destroying pre-trained knowledge.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# train the fine tuned model 
model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 53s 308ms/step - accuracy: 0.9579 - loss: 0.1138 - val_accuracy: 0.9514 - val_loss: 0.2030
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 52s 319ms/step - accuracy: 0.9756 - loss: 0.0661 - val_accuracy: 0.9658 - val_loss: 0.1340
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 53s 326ms/step - accuracy: 0.9833 - loss: 0.0458 - val_accuracy: 0.9712 - val_loss: 0.1120
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 54s 332ms/step - accuracy: 0.9888 - loss: 0.0337 - val_accuracy: 0.9712 - val_loss: 0.1183
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 55s 337ms/step - accuracy: 0.9883 - loss: 0.0328 - val_accuracy: 0.9658 - val_loss: 0.1477


### fine tuning evaluation performance

In [7]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")


36/36 ━━━━━━━━━━━━━━━━━━━━ 7s 192ms/step - accuracy: 0.9724 - loss: 0.1248
Test Accuracy: 0.9724


### **model daignoises** 
Even unfreezing 20 layers:
- Introduces more trainable parameters
- Small gradient noise slightly disturbs pretrained weights
- accuracy was same as before but loss increased

## **report**
“Fine-tuning did not improve performance because the frozen pretrained features already generalized well. Additional fine-tuning introduced minor noise, increasing loss without accuracy gain, so the  **frozen backbone model** was selected for **deployment** snice its has better generalization”.